In [21]:
%pylab inline
import os
import csv
import sqlite3 as lite
import math

import warnings
warnings.filterwarnings('ignore')

# numerical processing and scientific libraries
import numpy as np
import scipy

# signal processing
from scipy.io                     import wavfile
from scipy                        import stats, signal
from scipy.fftpack                import fft

from scipy.signal                 import lfilter, hamming
from scipy.fftpack.realtransforms import dct
from scikits.talkbox              import segment_axis
from scikits.talkbox.features     import mfcc

# general purpose
import collections

# plotting
import matplotlib.pyplot as plt
from numpy.lib                    import stride_tricks

from IPython.display              import HTML
from base64                       import b64encode

# Classification and evaluation
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.cross_validation import StratifiedKFold, ShuffleSplit, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

import pandas as pd
import pandas.io.sql as sql

import weka.core.jvm as jvm
from weka.core.converters import Loader, Saver
import weka.core.converters as converters

#Pooja's directories:
#PATH_WAV="F:/set1_wav/"
#PATH_DATASET="F:/set1/"
#wekaFilePackage= 'C:\Users\Pooja Bhojwani\wekafiles\packages'


#Arturo's Directories:
PATH_WAV="C:/Users/Cristina/Documents/MIR/Project/DataSet/set1/set1_wav/"
PATH_DATASET="C:/Users/Cristina/Documents/MIR/Project/DataSet/set1/set1/"
wekaFilePackage ='C:\Users\Cristina\wekafiles\packages'

DATASET="mean_ratings_set1.csv"
DB="CSC_479_MusicInformationRetrieval.db"

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [22]:
def dbInit():
    connection = lite.connect(DB)
    
    with connection:
        cursor = connection.cursor() 
        cursor.execute("DROP TABLE IF EXISTS musicFeatures")
        #Add features
        cursor.execute("CREATE TABLE musicFeatures( \
          id                 TEXT,\
          song               TEXT,\
          path               TEXT,\
          valence            TEXT,\
          energy             TEXT,\
          tension            TEXT,\
          sc_mean            TEXT,\
          sc_std             TEXT,\
          sc_var             TEXT,\
          rms_mean           TEXT,\
          rms_std            TEXT,\
          rms_var            TEXT,\
          sr_mean            TEXT,\
          sr_std             TEXT,\
          sr_var             TEXT,\
          zcr_mean           TEXT,\
          zcr_std            TEXT,\
          zcr_var            TEXT,\
          sf_mean            TEXT,\
          sf_std             TEXT,\
          sf_var             TEXT,\
          pe_mean            TEXT,\
          pe_std             TEXT,\
          pe_var             TEXT,\
          mfcc_mean          TEXT,\
          mfcc_std           TEXT,\
          mfcc_var           TEXT,\
          class              TEXT \
      )")
        
    print("Table created")
       

In [23]:
def dbInsert(songInfo):
    connection = lite.connect(DB)
    with connection:
        cursor= connection.cursor()
        #Add features
        cursor.executemany("INSERT INTO musicFeatures VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,\
                                                             ?, ?, ?, ?, ?, ?, ?, ?)", songInfo)

In [24]:
def convertToCSV(path):
    connection = lite.connect(DB)
    
    #table = sql.read_frame('select id, song, path, valence, energy, tension, sc_mean, sc_std, sc_var, rms_mean, rms_std, rms_var,  \
    #                               sr_mean, sr_std, sr_var, zcr_mean, zcr_std, zcr_var, sf_mean, sf_std, sf_var, \
    #                               mfcc_mean, mfcc_std, mfcc_var, class \
    #                        from musicFeatures', connection)
    dataset=path+'dataset.csv'
    #dataset_new= path+'dataset_new.csv'
    #the_frame = sql.read_sql_table('musicFeatures', connection)
    #table.to_csv(dataset)
    cursor = connection.cursor()
    cursor.execute("select * from musicFeatures;")

    with open(dataset, "wb") as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow([i[0] for i in cursor.description]) # write headers
        csv_writer.writerows(cursor)

In [25]:
def convertToARFF(path):
   
    #jvm.start(class_path = wekaFilePackage)
    jvm.start()    
    loader = Loader(classname="weka.core.converters.CSVLoader")
    data = loader.load_file(path + 'dataset.csv')
    #print(data)
    saver = Saver(classname="weka.core.converters.ArffSaver")
    saver.save_file(data, path + 'dataset.arff')

In [26]:
""" short time fourier transform of audio signal """
def stft(sig, frameSize, overlapFac=0.5, window=np.hanning):
    win = window(frameSize)
    hopSize = int(frameSize - np.floor(overlapFac * frameSize))
    
    # zeros at beginning (thus center of 1st window should be for sample nr. 0)
    samples = np.append(np.zeros(np.floor(frameSize/2.0)), sig)    
    # cols for windowing
    cols = np.ceil( (len(samples) - frameSize) / float(hopSize)) + 1
    # zeros at end (thus samples can be fully covered by frames)
    samples = np.append(samples, np.zeros(frameSize))
    
    frames = stride_tricks.as_strided(samples, shape=(cols, frameSize), strides=(samples.strides[0]*hopSize, samples.strides[0])).copy()
    frames *= win
    
    return np.fft.rfft(frames)

In [27]:
""" plot spectrogram"""
def plotstft(samples, samplerate, binsize=2**10, plotpath=None, colormap="jet", ax=None, fig=None):
    
    s = stft(samples, binsize)
    
    sshow, freq = logscale_spec(s, factor=1.0, sr=samplerate)
    ims = 20.*np.log10(np.abs(sshow)/10e-6) # amplitude to decibel
    
    timebins, freqbins = np.shape(ims)
    
    if ax is None:
        fig, ax = plt.subplots(1, 1, sharey=True, figsize=(PLOT_WIDTH, 3.5))
    
    #ax.figure(figsize=(15, 7.5))
    cax = ax.imshow(np.transpose(ims), origin="lower", aspect="auto", cmap=colormap, interpolation="none")
    #cbar = fig.colorbar(cax, ticks=[-1, 0, 1], cax=ax)
    #ax.set_colorbar()

    ax.set_xlabel("time (s)")
    ax.set_ylabel("frequency (hz)")
    ax.set_xlim([0, timebins-1])
    ax.set_ylim([0, freqbins])

    xlocs = np.float32(np.linspace(0, timebins-1, 5))
    ax.set_xticks(xlocs, ["%.02f" % l for l in ((xlocs*len(samples)/timebins)+(0.5*binsize))/samplerate])
    ylocs = np.int16(np.round(np.linspace(0, freqbins-1, 10)))
    ax.set_yticks(ylocs, ["%.02f" % freq[i] for i in ylocs])
    
    if plotpath:
        plt.savefig(plotpath, bbox_inches="tight")
    else:
        plt.show()
        
    #plt.clf();
    b = ["%.02f" % l for l in ((xlocs*len(samples)/timebins)+(0.5*binsize))/samplerate]
    return xlocs, b, timebins

In [28]:
""" scale frequency axis logarithmically """    
def logscale_spec(spec, sr=44100, factor=20.):
    timebins, freqbins = np.shape(spec)

    scale = np.linspace(0, 1, freqbins) ** factor
    scale *= (freqbins-1)/max(scale)
    scale = np.unique(np.round(scale))
    
    # create spectrogram with new freq bins
    newspec = np.complex128(np.zeros([timebins, len(scale)]))
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            newspec[:,i] = np.sum(spec[:,scale[i]:], axis=1)
        else:        
            newspec[:,i] = np.sum(spec[:,scale[i]:scale[i+1]], axis=1)
    
    # list center freq of bins
    allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./sr)[:freqbins+1])
    freqs = []
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            freqs += [np.mean(allfreqs[scale[i]:])]
        else:
            freqs += [np.mean(allfreqs[scale[i]:scale[i+1]])]
    
    return newspec, freqs

In [29]:
def spectralCentroid(wavedata, window_size, sample_rate):
    magnitude_spectrum = stft(wavedata, window_size)
    timebins, freqbins = np.shape(magnitude_spectrum)
    # when do these blocks begin (time in seconds)?
    timestamps = (np.arange(0,timebins - 1) * (timebins / float(sample_rate)))
    sc = []

    for t in range(timebins-1):
        power_spectrum = np.abs(magnitude_spectrum[t])**2
        sc_t = np.sum(power_spectrum * np.arange(1,freqbins+1)) / np.sum(power_spectrum)
        sc.append(sc_t)
        
    sc = np.asarray(sc)
    sc = np.nan_to_num(sc)
    
    return sc, np.asarray(timestamps)

In [30]:
def plotFeatureSpectogram(feature_data, timestamps, squared_wf=False):
    # plot spectrogram
    a,b,c = plotstft(sound_files["wavedata"], sound_files["samplerate"]);

    fig = plt.figure(num=None, figsize=(PLOT_WIDTH, PLOT_HEIGHT), dpi=72, facecolor='w', edgecolor='k');
    channel_1 = fig.add_subplot(111);
    channel_1.set_ylabel('Channel 1');
    channel_1.set_xlabel('time');

    # plot waveform
    scaled_wf_y = ((np.arange(0,sound_files["wavedata"].shape[0]).astype(np.float)) / sound_files["samplerate"]) * 1000.0
    
    if squared_wf:
        scaled_wf_x = (sound_files["wavedata"]**2 / np.max(sound_files["wavedata"]**2))
    else:
        scaled_wf_x = (sound_files["wavedata"] / np.max(sound_files["wavedata"]) / 2.0 ) + 0.5
    
    #scaled_wf_x = scaled_wf_x**2
    
    plt.plot(scaled_wf_y, scaled_wf_x, color='lightgrey');

    # plot feature-data
    scaled_fd_y = timestamps * 1000.0
    scaled_fd_x = (feature_data / np.max(feature_data))
    
    plt.plot(scaled_fd_y, scaled_fd_x, color='r');

    plt.show();
    plt.clf();

In [31]:
def rms(wavedata, block_length, sample_rate):
    # how many blocks have to be processed?
    num_blocks = int(np.ceil(len(wavedata)/block_length))
    # when do these blocks begin (time in seconds)?
    # timestamps = (np.arange(0,num_blocks - 1) * (block_length / float(samplerate)))
    rms = []
    for i in range(0,num_blocks-1):
        start = i * block_length
        stop  = np.min([(start + block_length - 1), len(wavedata)])
        rms_seg = float(np.sqrt(np.mean(wavedata[start:stop]**2)))
        x=float('nan')
        if not math.isnan(rms_seg):
            rms.append(rms_seg)
        #print(rms)
    return np.asarray(rms)

In [32]:
def spectralRolloff(wavedata, window_size, sample_rate, k=0.85):    
    # convert to frequency domain
    magnitude_spectrum = stft(wavedata, window_size)
    power_spectrum     = np.abs(magnitude_spectrum)**2
    timebins, freqbins = np.shape(magnitude_spectrum)
    # when do these blocks begin (time in seconds)?
    #timestamps = (np.arange(0,timebins - 1) * (timebins / float(sample_rate)))
    
    sr = []

    spectralSum    = np.sum(power_spectrum, axis=1)
    
    for t in range(timebins-1):
        # find frequency-bin indeces where the cummulative sum of all bins is higher
        # than k-percent of the sum of all bins. Lowest index = Rolloff
        sr_t = np.where(np.cumsum(power_spectrum[t,:]) >= k * spectralSum[t])[0][0]
        sr.append(sr_t)
    sr = np.asarray(sr).astype(float)
    
    # convert frequency-bin index to frequency in Hz
    sr = (sr / freqbins) * (sample_rate / 2.0)
    
    return sr

In [33]:
def zero_crossing_rate(wavedata, block_length, sample_rate):    
    # how many blocks have to be processed?
    num_blocks = int(np.ceil(len(wavedata)/block_length))
    # when do these blocks begin (time in seconds)?
    timestamps = (np.arange(0,num_blocks - 1) * (block_length / float(samplerate)))
    
    zcr = []
    
    for i in range(0,num_blocks-1):        
        start = i * block_length
        stop  = np.min([(start + block_length - 1), len(wavedata)])        
        zc = 0.5 * np.mean(np.abs(np.diff(np.sign(wavedata[start:stop]))))
        zcr.append(zc)
        
    return np.asarray(zcr), np.asarray(timestamps)

In [34]:
def spectral_flux(wavedata, window_size, sample_rate):    
    # convert to frequency domain
    magnitude_spectrum = stft(wavedata, window_size)
    timebins, freqbins = np.shape(magnitude_spectrum)    
    # when do these blocks begin (time in seconds)?
    timestamps = (np.arange(0,timebins - 1) * (timebins / float(sample_rate)))
    
    sf = np.sqrt(np.sum(np.diff(np.abs(magnitude_spectrum))**2, axis=1)) / freqbins
    
    return sf[1:], np.asarray(timestamps)

In [35]:
def pitch_extract(file):
    import subprocess
    os.chdir(PATH_WAV)
    subprocess.check_output(PATH_WAV + '/pitchextract.exe -w 1024 -l 3 -u 256 ' + PATH_WAV + file, shell=True)
    pitchpath = '\pitch.txt'
    os.chdir(PATH_WAV)
    cwd = os.getcwd()
    #print cwd, pitchpath
    pitchfile = cwd + pitchpath
    f = open(pitchfile, 'r')
    #print "File" + file
    lines = [line.rstrip('\n') for line in f]
    results = [float(i) for i in lines]
    #temp = list()
    #for line in lines:
    #    if isinstance(line, str):
    #        line = float(line)
            #print line,type(line)
    return results
    #return lines

In [36]:
def mfcc_extract(wavedata):
        # Parameters
        nwin    = 256
        nfft    = 1024
        fs      = 16000
        nceps   = 13

        # Pre-emphasis factor (to take into account the -6dB/octave
        # rolloff of the radiation at the lips level)
        prefac  = 0.97

        # MFCC parameters: taken from auditory toolbox
        over    = nwin - 160

        filtered_data = lfilter([1., -prefac], 1, wavedata)
        
        windows     = hamming(256, sym=0)
        framed_data = segment_axis(filtered_data, nwin, over) * windows

        # Compute the spectrum magnitude
        magnitude_spectrum = np.abs(fft(framed_data, nfft, axis=-1))
        
        
        # Compute triangular filterbank for MFCC computation.

        lowfreq  = 133.33
        linsc    = 200/3.
        logsc    = 1.0711703
        fs = 44100

        nlinfilt = 13
        nlogfilt = 27

        # Total number of filters
        nfilt    = nlinfilt + nlogfilt

        #------------------------
        # Compute the filter bank
        #------------------------
        # Compute start/middle/end points of the triangular filters in spectral
        # domain
        freqs            = np.zeros(nfilt+2)
        freqs[:nlinfilt] = lowfreq + np.arange(nlinfilt) * linsc
        freqs[nlinfilt:] = freqs[nlinfilt-1] * logsc ** np.arange(1, nlogfilt + 3)
        heights          = 2./(freqs[2:] - freqs[0:-2])

        # Compute filterbank coeff (in fft domain, in bins)
        filterbank  = np.zeros((nfilt, nfft))

        # FFT bins (in Hz)
        nfreqs = np.arange(nfft) / (1. * nfft) * fs

        for i in range(nfilt):
    
            low = freqs[i]
            cen = freqs[i+1]
            hi  = freqs[i+2]
            lid    = np.arange(np.floor(low * nfft / fs) + 1,
                       np.floor(cen * nfft / fs) + 1, dtype=np.int)
            rid    = np.arange(np.floor(cen * nfft / fs) + 1,
                       np.floor(hi * nfft / fs)  + 1, dtype=np.int)
            lslope = heights[i] / (cen - low)
            rslope = heights[i] / (hi - cen)
            filterbank[i][lid] = lslope * (nfreqs[lid] - low)
            filterbank[i][rid] = rslope * (hi - nfreqs[rid])
        
        # apply filter
        mspec = np.log10(np.dot(magnitude_spectrum, filterbank.T))
        
        # Compute triangular filterbank for MFCC computation.

        lowfreq  = 133.33
        linsc    = 200/3.
        logsc    = 1.0711703
        fs = 44100

        nlinfilt = 13
        nlogfilt = 27

        # Total number of filters
        nfilt    = nlinfilt + nlogfilt

        #------------------------
        # Compute the filter bank
        #------------------------
        # Compute start/middle/end points of the triangular filters in spectral
        # domain
        freqs            = np.zeros(nfilt+2)
        freqs[:nlinfilt] = lowfreq + np.arange(nlinfilt) * linsc
        freqs[nlinfilt:] = freqs[nlinfilt-1] * logsc ** np.arange(1, nlogfilt + 3)
        heights          = 2./(freqs[2:] - freqs[0:-2])

        # Compute filterbank coeff (in fft domain, in bins)
        filterbank  = np.zeros((nfilt, nfft))

        # FFT bins (in Hz)
        nfreqs = np.arange(nfft) / (1. * nfft) * fs
        
        for i in range(nfilt):
                
                low = freqs[i]
                cen = freqs[i+1]
                hi  = freqs[i+2]
                
                lid    = np.arange(np.floor(low * nfft / fs) + 1,
                           np.floor(cen * nfft / fs) + 1, dtype=np.int)
                
                rid    = np.arange(np.floor(cen * nfft / fs) + 1,
                           np.floor(hi * nfft / fs)  + 1, dtype=np.int)
                
                lslope = heights[i] / (cen - low)
                rslope = heights[i] / (hi - cen)
                
                filterbank[i][lid] = lslope * (nfreqs[lid] - low)
                filterbank[i][rid] = rslope * (hi - nfreqs[rid])
        
        # apply filter
        mspec = np.log10(np.dot(magnitude_spectrum, filterbank.T))
        
        #fig, ax = plt.subplots(2, 1, sharey=True, figsize=(PLOT_WIDTH, 5.5))
        #cax = ax[0].imshow(np.transpose(magnitude_spectrum), origin="lower", aspect="auto",  interpolation="nearest")
        #cax = ax[1].imshow(np.transpose(mspec), origin="lower", aspect="auto",  interpolation="nearest")
        plt.show()
        ## Use the DCT to 'compress' the coefficients (spectrum -> cepstrum domain)
        MFCCs = dct(mspec, type=2, norm='ortho', axis=-1)[:, :nceps]
        MFCCs, mspec, spec = mfcc(wavedata)
        return MFCCs
        
        #fig, ax = plt.subplots(1, 1, sharey=True, figsize=(PLOT_WIDTH, 3.5))
        #cax = ax.imshow(np.transpose(MFCCs), origin="lower", aspect="auto",  interpolation="nearest")
        #plt.show()

In [37]:
def metrics(data, feature, aggregators):   
    for aggr_name in aggregators:        
        if aggr_name == "mean":
            mean=np.mean(data[feature])
            #print("mean: %f " % mean)
        elif aggr_name == "std":
            std=np.std(data[feature])
            #print("Standard Deviation: %f" % std)
        elif aggr_name == "var":
            var=np.var(data[feature])
            #print("Variance: %f" % var)
    return mean,std, var    

In [38]:
def printMetrics(mean, std, var):
    print("------------------------------------")
    print("           Metrics                  ")
    print("------------------------------------")
    print(" Mean     |   Standard D  |  Variance  ")
    print("%f     %f      %f     "%(mean,std,var))

In [ ]:
def readCSV(file_name,id,sound_files):
    #['1', '4.83', '6.83', '3.17', '1.00', '1.00', '7.33', '1.00', '1.00', 'HAPPY', '', '']
    with open (file_name) as f:
        #print(file_name)
        readCSV = csv.reader (f, delimiter=',')
        readCSV.next()
        for row in readCSV:
            songId= int(row[0])
            if (songId==id):
                valence=row[1]
                energy=row[2]
                tension=row[3]
                classe=row[9]
                return valence, energy, tension, classe

In [ ]:
#defaultdict means that if a key is not found in the dictionary, then instead of a KeyError being thrown, 
#a new entry is created.
# initialize music collection
dbInit()
sound_files = collections.defaultdict(dict)

for filename in sorted(os.listdir(PATH_WAV)):
    if filename.endswith(".wav"):
        song=PATH_WAV+filename
        samplerate, wavedata = wavfile.read(song)
        id=filename[0:-4]
        
        sound_files["path"]= song
        sound_files["id"]=id
        sound_files["samplerate"]        = samplerate
        sound_files["wavedata"]          = wavedata
        sound_files["number_of_samples"] = wavedata.shape[0]
        sound_files["song_length"]       = int( sound_files["number_of_samples"] / samplerate ) # round up
                
        sound_files["sc"]= spectralCentroid(sound_files["wavedata"], 1024, sound_files["samplerate"])
        print("Song: %s" % song)
        #print(sound_files)
        sound_files["sc_mean"],sound_files["sc_std"],sound_files["sc_var"]= metrics(sound_files, "sc", ["mean", "std","var"])
        #printMetrics(mean,std,var)
        
        sound_files["rms"]= rms(sound_files["wavedata"], 2048, sound_files["samplerate"]);
        #print(sound_files["rms"])
        sound_files["rms_mean"],sound_files["rms_std"],sound_files["rms_var"]= metrics(sound_files, "rms", ["mean", "std","var"])
        #printMetrics(mean,std,var)
        
        sound_files["sr"]= spectralRolloff(sound_files["wavedata"],1024, sound_files["samplerate"],k=0.85)
        sound_files["sr_mean"],sound_files["sr_std"],sound_files["sr_var"]= metrics(sound_files, "sr", ["mean", "std","var"])
        #printMetrics(mean,std,var)
        
        sound_files["zcr"]= zero_crossing_rate(sound_files["wavedata"], 2048, sound_files["samplerate"]);
        sound_files["zcr_mean"],sound_files["zcr_std"],sound_files["zcr_var"]= metrics(sound_files, "zcr", ["mean", "std","var"])
        #printMetrics(mean,std,var)
        
        sound_files["sf"]= spectral_flux(sound_files["wavedata"], 1024, sound_files["samplerate"]);
        sound_files["sf_mean"],sound_files["sf_std"],sound_files["sf_var"]= metrics(sound_files, "sf", ["mean", "std","var"])
        #printMetrics(mean,std,var)
        
        sound_files["pe"] = pitch_extract(filename)
        #pitch_extract(filename)
        #print metrics(sound_files, "pe", ["mean", "std","var"])
        sound_files["pe_mean"],sound_files["pe_std"],sound_files["pe_var"]= metrics(sound_files, "pe", ["mean", "std","var"])
        #mean,std,var= metrics(sound_files, "pe", ["mean", "std","var"])
        #printMetrics(mean,std,var)
        
        sound_files["mfcc"]= mfcc_extract(sound_files["wavedata"])
        sound_files["mfcc_mean"],sound_files["mfcc_std"],sound_files["mfcc_var"]= metrics(sound_files, "mfcc", ["mean", "std","var"])
        #printMetrics(mean,std,var)
        #print sound_files["mfcc_mean"],sound_files["mfcc_std"],sound_files["mfcc_var"]
        
        dataset=PATH_DATASET+DATASET
        ID=float(id)
        sound_files["valence"],sound_files["energy"],sound_files["tension"],sound_files["class"]=readCSV(dataset,ID,sound_files)
        
        songInfo = [(str(sound_files["id"]),str(filename),str(sound_files["path"]),\
                     str(sound_files["valence"]), str(sound_files["energy"]),str(sound_files["tension"]),\
                     str(sound_files["sc_mean"]), str(sound_files["sc_std"]),str(sound_files["sc_var"]),\
                     str(sound_files["rms_mean"]), str(sound_files["sc_std"]),str(sound_files["sc_var"]),\
                     str(sound_files["sr_mean"]), str(sound_files["sr_std"]),str(sound_files["sr_var"]), \
                     str(sound_files["zcr_mean"]), str(sound_files["zcr_std"]),str(sound_files["zcr_var"]),\
                     str(sound_files["sf_mean"]), str(sound_files["sf_std"]),str(sound_files["sf_var"]),\
                     str(sound_files["pe_mean"]), str(sound_files["pe_std"]),str(sound_files["pe_var"]),\
                     str(sound_files["mfcc_mean"]), str(sound_files["mfcc_std"]),str(sound_files["mfcc_var"]),\
                     str(sound_files["class"]))]
       
        #print(songInfo)
        dbInsert(songInfo)
        convertToCSV(PATH_DATASET)
        convertToARFF(PATH_DATASET)
        
        
        #plotFeatureSpectogram(sc, ts, squared_wf=False);
        #songInfo = [(str(id),str(filename),str(path),str(sc),str(zcr),str(sf),)]
        #dbInsert(songInfo)        

Table created
Song: C:/Users/Cristina/Documents/MIR/Project/DataSet/set1/set1_wav/001.wav
Song: C:/Users/Cristina/Documents/MIR/Project/DataSet/set1/set1_wav/002.wav